# Galaxies
## Introduction to Astronomy, AGS Winterim 2023
By: Mathilda Avirett-Mackenzie



In [ ]:
# packages we will use
import numpy as np                     # needed for some calculations
import matplotlib.pyplot as plt        # does plotting
from astropy.table import Table        # to read in data tables
from astropy.cosmology import Planck18 # to calculate distances

## Galaxy Data
We will use magnitude and position data of galaxies in the [Sloan Digital Sky Survey](https://sdss.org), retrieved with their [SQL search tool](http://skyserver.sdss.org/dr17/SearchTools/sql).

The $u$, $g$, $r$, $i$, and $z$ columns correspond to the different [filters](https://skyserver.sdss.org/dr1/en/proj/advanced/color/sdssfilters.asp) used by the SDSS camera. Filters restrict the image to a certain range of wavelengths (kind of like how our eyes can only see light between about 4000 and 7000 [Angstroms](https://en.wikipedia.org/wiki/Angstrom)). The filters are:

| Filter    | Range         | Peak Wavelength |
| :-:       | :-:           | :-:             |
| $u$       | ultraviolet   | 3543 A          |
| $g$       | green         | 4770 A          |
| $r$       | red           | 6231 A          |
| $i$       | near infrared | 7625 A          |
| $z$       | far infrared  | 9134 A          |

In [ ]:
gal_cat = Table.read('../data/sdss-galaxies.csv', format='ascii')

In [ ]:
gal_cat

## Color vs magnitude
Like the Hertzsprung-Russell diagram for stars, we can characterize galaxies by plotting color vs brightness. As a reminder, in astronomy we express color as flux ratio or magnitude difference of red light to blue light. A larger color value means a source is redder, while a smaller color value means it is bluer.

In [ ]:
color = gal_cat['g'] - gal_cat ['r'] # have to calculate this ourselves this time

We can immediately see that the galaxies are more likely to have colors around 0.5 or 0.9 and less likely to fall in the middle. This is called a bimodal distribution.

In [ ]:
# calculate absolute magnitude from distance
# distances to galaxies can get a bit complicated as they are moving away from us
distance = Planck18.luminosity_distance(gal_cat['redshift']).value*1e6
magnitude = gal_cat['r'] - 5*np.log10(distance) + 5   # using the definition of absolute magnitude

In [ ]:
fig = plt.figure(figsize=(20,20))   # create a figure
ax = fig.subplots(nrows=1, ncols=1) # create the plot on the figure

# scatter plot of colors and magnitudes
ax.scatter(magnitude, color, c='k', s=1, zorder=5, alpha=0.1) 

# labelling axes
ax.set_ylabel(r'$g-r$ color', size=24)
ax.set_xlabel(r'$r$-band magnitude', size=24)

# formatting stuff, don't bother reading this bit
# ax.set_yscale('log')
ax.grid('True', zorder=-2) 
ax.set_xlim(-16, -24)
ax.set_ylim(0,1.2)
ax.tick_params(which='minor', left=True, right=True, direction='in')
ax.tick_params(which='major', left=True, right=True, bottom=True, top=True, direction='in', labelsize=20)

There's lots of outliers, but we can see a clear red sequence with $g-r$ between 0.8 and 1, and a somewhat more diffuse blue cloud in the general region of $g-r$ between 0.4 and 0.7.

We can grab the coordinates of some typical galaxies from both regions and look them up in [SkyServer](https://skyserver.sdss.org) to see what they look like. Try looking at both the images and spectra for a few red galaxies and a few blue galaxies.

You may find you'll need to be accurate to up to 3 decimal places in the coordinates to land on the galaxy in SkyServer. Tick the "Objects with spectra" box on the left side under "Drawing Options" to make it draw a red box around all galaxies with spectra.

In [ ]:
# red sequence
gal_cat[['ra', 'dec']][np.logical_and(np.logical_and(color > 0.85, color < 0.95),
                                      np.logical_and(magnitude < -20.5, magnitude > -21.5))]

In [ ]:
# blue cloud
gal_cat[['ra', 'dec']][np.logical_and(np.logical_and(color > 0.4, color < 0.6),
                                      np.logical_and(magnitude < -20, magnitude > -20.5))]

## Sky position
Galaxies aren't evenly spaced in the universe; rather, they form a web-like pattern of filaments and nodes surrounding big empty voids. We can see this in action just by plotting the sky positions of galaxies at the same redshift.

In [ ]:
fig = plt.figure(figsize=(20,20))   # create a figure
ax = fig.subplots(nrows=1, ncols=1) # create the plot on the figure

# scatter plot of positions
ax.scatter(gal_cat['ra'], gal_cat['dec'], c='k', s=1, zorder=5, alpha=0.1) 

# labelling axes
ax.set_xlabel('Right Ascension (deg)', size=24)
ax.set_ylabel('Declination (deg)', size=24)

# formatting stuff, don't bother reading this bit
# ax.set_yscale('log')
ax.grid('True', zorder=-2) 
ax.set_xlim(175, 225)
ax.set_ylim(10, 60)
ax.tick_params(which='minor', left=True, right=True, direction='in')
ax.tick_params(which='major', left=True, right=True, bottom=True, top=True, direction='in', labelsize=20)